# Vector to Vector Squared Network & Training

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import collections
from collections import OrderedDict

## User Inputs

In [ ]:
infile = 'vtov2_dataset.npz'
batch_size = 250
n_epochs = 500
hidden_layers = [(1,50)]
valid_size = 2000
model_path = 'vtov2_model_1'
fig_path = 'vtov2_fig_1.png'
fig2_path = 'vtov2_fig_2.png'
df_path = 'vtov2_df_1'
seed = np.random.randint(low=0, high=1000000, size=1)

## Loading Data

In [ ]:
npfile = np.load(infile)
inputs = npfile['inputs']
outputs = npfile['outputs']

In [ ]:
# standardizing inputs, outputs and coverting to tensors
inputMeans = inputs[0:int(inputs.shape[0]),:].mean(axis=0)
inputStdDevs = inputs[0:int(inputs.shape[0]),:].std(axis=0)
inputs = (inputs-inputMeans)/inputStdDevs
inputs = torch.from_numpy(inputs).float()

outputMeans = outputs[0:int(outputs.shape[0]),:].mean(axis=0)
outputStdDevs = outputs[0:int(outputs.shape[0]),:].std(axis=0)
outputs = (outputs-outputMeans)/outputStdDevs
outputs = torch.from_numpy(outputs).float()

In [ ]:
# generating a TensorDataset for training
trainset = torch.utils.data.TensorDataset(inputs, outputs)

## Network Definition

In [ ]:
# input, output sizes
out_size = list(outputs[0].size())[0]
in_size = list(inputs[0].size())[0]

# number of hidden layers
num = len(hidden_layers)

# number of nodes in a given hidden layer
def nodes(i):
    layer = hidden_layers[i]
    dim_node = layer[1]
    return dim_node

In [ ]:
network = OrderedDict([])

network = OrderedDict([('lin1', nn.Linear(in_size, nodes(0))),('relu1', nn.PReLU())]) 
if num > 1:
    for i in range(1, num):
        network['lin{index}'.format(index=i+1)] = nn.Linear(nodes(i-1), nodes(i))
        network['relu{index}'.format(index=i+1)] = nn.PReLU()
network['lin{index}'.format(index=num+1)] = nn.Linear(nodes(num-1), out_size)    

model = nn.Sequential(network)

## Function Definitions

In [ ]:
# defines the training process for one epoch, returns training loss for given epoch
def train(model, loader, optimizer, criterion):
    running_train_loss = 0.0

    # put model into train mode
    model.train()

    for batch_idx, (inputs, outputs) in enumerate(loader):
        inputs_var = inputs
        outputs_var = outputs
        
        # get model output & loss for each given input
        model_outputs = model(inputs_var)
        loss = criterion(model_outputs, outputs_var)

        # record cummulative loss
        running_train_loss += loss.item()

        # gradient, optimizer steps
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return running_train_loss


# defines the validation process for one epoch, returns validation loss for given epoch
def validate(model, loader, criterion):
    running_valid_loss = 0.0

    # put model in evaluation mode
    model.eval()

    for batch_idx, (inputs, outputs) in enumerate(loader):
        with torch.no_grad():
            inputs_var = inputs
            outputs_var = outputs

            # get model output & loss for each given input
            model_outputs = model(inputs_var)
            loss = criterion(model_outputs, outputs_var)

        # record cummulative loss
        running_valid_loss += loss.item()

    return running_valid_loss


# runs training and validation process over all epochs, returns results
def run_training(model, modelpath, figpath, fig2path, dfpath, trainset, validsize, numepochs, batchsize, seed):
    # set seed
    torch.manual_seed(seed)

    # create validation split
    indices = torch.randperm(len(trainset))
    train_indices = indices[:len(indices) - valid_size]
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
    valid_indices = indices[len(indices) - valid_size:]
    valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_indices)

    # define data loaders
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=valid_sampler)
  
    # set criterion, optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    # store results
    best_model = copy.deepcopy(model.state_dict())
    train_loss_results = []
    valid_loss_results = []
    epochs = []
    
    # train model
    for epoch in enumerate(range(n_epochs)):
        trainloss = train(model=model,loader=train_loader,criterion=criterion,optimizer=optimizer)
        print('train loss for epoch {index} attained: {loss}'.format(index=epoch[0], loss=trainloss))
        
        validloss = validate(model=model,loader=valid_loader,criterion=criterion)
        print('valid loss for epoch {index} attained: {loss}'.format(index=epoch[0], loss=validloss))
        
        train_loss_results.append(trainloss)
        valid_loss_results.append(validloss)
        epochs.append(epoch[0]+1)
        
        # check if model is the best, save if best
        if epoch[0] == 0:
            bestloss = validloss

        if epoch[0] > 0:
            if validloss < bestloss:
                bestloss = validloss
                best_model = copy.deepcopy(model)
                best_epoch = epoch[0]
                print('new best model saved')
                
        print('epoch {index} done'.format(index=epoch[0]))
        
    print('finished looping epochs')
    print('best valid loss = {}, epoch {}'.format(bestloss, best_epoch))

    # load and save the best model
    torch.save(best_model, model_path)
    print('best model loaded and saved')

    # plot training & validation loss vs. epoch
    fig1 = plt.figure()
    plt.plot(epochs, train_loss_results)
    plt.plot(epochs, valid_loss_results)
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper left')
    plt.title('Model Training and Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    fig1.savefig(fig_path)
    print('plot saved')
    
    # plot training & validation loss vs. epoch -- scale 2
    fig2 = plt.figure()
    plt.plot(epochs, train_loss_results)
    plt.plot(epochs, valid_loss_results)
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper left')
    plt.title('Model Training and Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.ylim(0,(bestloss*100))
    plt.show()
    fig2.savefig('fig2_path')
    print('plot saved')
    
    # create dataframe of epochs, losses
    d = {'trainloss':train_loss_results, 'validloss':valid_loss_results}
    df = pd.DataFrame(d, index=epochs)
    df.to_csv(df_path)
    print('dataframe saved')
    
    return

## Run Training Function

In [ ]:
# train the model
run_training(model=model, modelpath=model_path, figpath=fig_path, fig2path = fig2_path, dfpath=df_path, trainset=trainset, 
      validsize=valid_size, numepochs=n_epochs, batchsize=batch_size, seed=seed)

print('Finished!')

In [ ]:
list(model.parameters())